# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по одному баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 02.10.24 23:59__

__Жесткий дедлайн: 05.10.24 23:59__


### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в телефонах, поисковой строке или приложении почты. Полученную систему вам нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex), чтобы ей можно было удобно пользоваться, а так же, чтобы убедиться, что все работает как надо. В этот раз вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей предположительно одинаковых по сложности. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы получилось в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание
При сдаче зададания в anytask вам будет необходимо сдать весь код, а также отчет с подробным описанием техник, которые в применили для создания вашей системы. Не лишним будет также написать и о том, что у вас не получилось и почему.

За часть с заданиями можно будет получить до __5__ баллов, за отчет – до __3__ баллов и еще __2__ балла можно будет получить за демонстрацию вашей системы и пользовательского интерфейса. Демонстрацию прикрепляйте в anytask в виде 1-2 минутной записи экрана.

## Часть 1

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [28]:
import pandas as pd
from typing import Union, List, Tuple
import re
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
import itertools

In [29]:
emails = pd.read_csv('emails.csv')
len(emails)

517401

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (1 балл).__ Очистите корпус текстов по вашему усмотрению. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст. Оценка будет выставляться по близости вашего результата к этому идеалу.

In [30]:
print(emails.iloc[550,1])

Message-ID: <10586305.1075855378285.JavaMail.evans@thyme>
Date: Mon, 7 May 2001 16:23:00 -0700 (PDT)
From: phillip.allen@enron.com
To: stanley.horton@enron.com, dmccarty@enron.com
Subject: California Summary
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: stanley.horton <stanley.horton@enron.com>, dmccarty <dmccarty@enron.com>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst


---------------------- Forwarded by Phillip K Allen/HOU/ECT on 05/07/2001 11:22 AM ---------------------------
   

	  From:  Jay Reitmeyer                           05/03/2001 11:03 AM	
		


To:	stanley.horton@enron.com, dmccarty@enron.com
cc:	 
Subject:	California Summary

Attached is the final version of the California Summary report with maps, graphs, and historical data.


 
To:	Phillip K Allen/HOU/ECT@ECT
cc:	 
bcc:	
Subject:	Additional California

Отсматривая сообщения руками, можно заметить, что метаданные отделяются от содержания* при помощи двойного переноса строки (\n\n).
Поскольку метаданные нам вообще не нужны, мы можем начать с того, что просто их откинем для простоты дальнейшего препроцессинга.
Да, двойной перенос строки будет еще в дальнейшем встречаться в тексте (особенно при переносе строк при обращениях внутри текста письма, например), но первый перенос строки точно предназначен для отделения метаданных.

Но этого недостаточно для того, чтобы полностью отделить лишнее: помимо метаданных у нас останется еще доп. информация по пересылке письма (дата, время, получатель, отправитель).

In [31]:
# отделяю содержание от метаданных по первому возникновению \n\n

emails['message_wo_metadata'] = emails['message'].apply(lambda x: x.split('\n\n', 1)[1])

In [32]:
print(emails.iloc[550,2])


---------------------- Forwarded by Phillip K Allen/HOU/ECT on 05/07/2001 11:22 AM ---------------------------
   

	  From:  Jay Reitmeyer                           05/03/2001 11:03 AM	
		


To:	stanley.horton@enron.com, dmccarty@enron.com
cc:	 
Subject:	California Summary

Attached is the final version of the California Summary report with maps, graphs, and historical data.


 
To:	Phillip K Allen/HOU/ECT@ECT
cc:	 
bcc:	
Subject:	Additional California Load Information



Additional charts attempting to explain increase in demand by hydro, load growth, and temperature.  Many assumptions had to be made.  The data is not as solid as numbers in first set of graphs.
 




Мы отделили содержание писем: теперь, если письмо не форвардится никуда, то мы по сути получили то, что хотели - просто текст (останется привести все в нижний регистр и убрать пунктуацию), но если оно куда-то перенаправляется, остается много мусора, который тоже нужно убрать.

Замечаем, что самое последнее, что после метаданных пересылки встречается перед непосредственным содержанием письма, это 'Subject: Theme \n\n'.

Используем это: отделим все, что находится левее самого последнего 'Subject: ', а потом отделим это по первому возникновению '\n\n', чтобы убрать тему, и получим текст без заголовка пересылки и самой пересылки.

(Я думаю, в пересылках оставлять сам текст письма, на который человек отвечает, не очень уместно, так как у нас должен на выходе получиться подсказчик текста, который подсказывает текст исходя из того, что это отдельная законченная мысль от одного человека, а сохраняя разные ответы в одном тексте письма может случиться так, что подсказчик будет выдавать продолжения в формате диалога, что будет странно. Поэтому мне кажется, что лучше просто само письмо оставлять в результате препроцессинга, без ответов/пересылок.)

In [33]:
print(emails.iloc[550,2])


---------------------- Forwarded by Phillip K Allen/HOU/ECT on 05/07/2001 11:22 AM ---------------------------
   

	  From:  Jay Reitmeyer                           05/03/2001 11:03 AM	
		


To:	stanley.horton@enron.com, dmccarty@enron.com
cc:	 
Subject:	California Summary

Attached is the final version of the California Summary report with maps, graphs, and historical data.


 
To:	Phillip K Allen/HOU/ECT@ECT
cc:	 
bcc:	
Subject:	Additional California Load Information



Additional charts attempting to explain increase in demand by hydro, load growth, and temperature.  Many assumptions had to be made.  The data is not as solid as numbers in first set of graphs.
 




In [34]:
# заметила, что применять описанный алгоритм на все строки плохо, тк сплит по \n\n выкинет часть письма, если в 
# сообщении не было темы, так что написала функцию, чтобы сплит по \n\n был только тогда, когда у нас есть тема (есть форвард)

def split_forward(msg):
    
    if 'Subject:' in msg:
        return msg.rsplit('Subject:', 1)[-1].split('\n\n', 1)[-1]
        
    else:
        return msg

In [35]:
emails['message_wo_forward'] = emails['message_wo_metadata'].apply(lambda x: split_forward(x))

In [36]:
print(emails.iloc[1003,3])

Thanks for the information.  It would be helpful if you would send the 
detailed worksheet that you mentioned.

I am surprised to hear that the only restricted shares left are the ones 
granted this January.  I have always elected to defer any distributions of 
restricted stock.  I believe I selected the minimum amount required to be 
kept in enron stock (50%).   Are you saying that all the previous grants have 
fully vested and been distributed to my deferral account?

Thank you for looking into this issue.

Phillip


Какие у нас еще остаются проблемы с текстом? Из того, что я смогла заметить: ссылки, назначенные в функционале почты встречи (тоже содержат ненужные метаданные, как пересылки), прикрепленные файлы, а также лишние \n, имейл адреса внутри писем, цифры (ими мы продолжать текст не будем, они будут только мешать - айпи адреса, время, даты, количества чего-то), пунктуация и спец сивмолы

Сообщений, приложенных к встречам, немного, и они обычно следуют перед пересылкой встречи с разделителем '----------------------'

In [37]:
emails['message_wo_meetings'] = emails['message_wo_forward'].apply(lambda x: x.split('----------------------', 1)[0])

Сообщения с большим количеством получателей (15-20+) форматируются по-другому, у них после первой итерации препроцессинга не убрались метаданные, поэтому я очищаю их повторно, тем же способом, что и форварды, то есть отсекая тему и \n\n:

In [38]:
emails['message_wo_subscription_metadata'] = emails['message_wo_meetings'].apply(lambda x: split_forward(x))

Отлично, все метаданные и детали функционала outlook вычищены (ответы, пересылки, встречи, детали массовой рассылки).
Теперь уберем частные загрязнения: имейл адреса внутри текста писем, приложенные файлы и ссылки

In [39]:
# remove email addresses
emails['message_wo_emails'] = emails['message_wo_subscription_metadata'].apply(lambda x: re.sub(r'\S*@\S*\s?', '', x))

# remove urls
emails['message_wo_urls'] = emails['message_wo_emails'].apply(lambda x: re.sub(r'http\S+', '', x))


# remove attached files
emails['message_wo_files'] = emails['message_wo_urls'].apply(lambda x: re.sub(r'-\s\S*.\S*', '', x))

Наша финальная итерация препроцессинга: убираем цифры, пунктуацию, приводим все к одному регистру

Обосную свое решение убрать пунктуацию тем, что с ней в целом будет сложно работать, и она будет создавать больше проблем, чем пользы:
знаки пунктуации не зависят от смысловой нагрузки слова (в отличие от n-грамм, где фокус идет на то, что связанные по смыслу слова часто встречаются вместе), они проставляются механическими правилами, наша система не сможет их вставлять к месту, она просто будет вставлять запятую и точку после слова, которое чаще всего предшествовало запятой (это скорее всего будет какая-нибудь служебная часть речи) или заканчивало предложение. Если же мы отнесемся к пунктуации не как к отдельному токену, а как к последнему символу токена предыдущего слова, у нас получится по 2 слова на одну словоформу, что увеличит время работы программы, пространство необходимое для хранения данных, но не добавит смысла, потому что пунктация проставляется не только на основе предшествующих слов, но и на основе следующих (для обозначения противопоставления, для отделения законченной мысли и так далее), а у нас не учитывается дальнейший контекст в функционале. Более того, мы можем жадным способом предсказывать слово за словом, и несмотря на схожесть контекста и смысла токенов "Х" и "Х,", в словаре сразу сузится количество возможных слов после "Х", потому что оно будет зависеть от того, как часто оно встречалось рядом именно с запятой. 


In [40]:
def preprocess(x):

    x_lowercase = x.lower()
    x_no_digits = re.sub('\d+', '', x_lowercase)
    x_no_nextstring = x_no_digits.replace('\n', ' ')
    x_no_punctuation = re.sub(r'[^a-zA-Z\s]', ' ', x_no_nextstring)
    x_final = re.sub(' +', ' ', x_no_punctuation)

    return x_final

In [41]:
emails['message_preprocessed'] = emails['message_wo_files'].apply(lambda x: preprocess(x))

In [42]:
for i in range(100):
    print(emails.iloc[i,9])
    print(i)

here is our forecast 
0
traveling to have a business meeting takes the fun out of the trip especially if you have to prepare a presentation i would suggest holding the business plan meetings here then take a trip without any formal business meetings i would even try and get some honest opinions on whether a trip is even desired or necessary as far as the business meetings i think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not too often the presenter speaks and the others are quiet just waiting for their turn the meetings might be better if held in a round table discussion format my suggestion for where to go is austin play golf and rent a ski boat and jet ski s flying somewhere takes too much time 
1
test successful way to go 
2
randy can you send me a schedule of the salary and level of everyone in the scheduling group plus your thoughts on any changes that need to be made patti s for example phillip
3
let

Ураа! Теперь нас есть финальные тексты сообщений без лишних символов и данных!

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы считаете, что результат получается лучше без нее, то можете удалить все небуквенные символы на этапе токенизации.

In [43]:
from nltk.tokenize import word_tokenize

emails['message_tokenized'] = emails['message_preprocessed'].apply(lambda x: word_tokenize(x))

## Дополнение слова

Описанная система будет состоять из двух частей: дополнение слова до целого и генерация продолжения текста (или вариантов продолжений). Начнем с первой части.

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова за $O(n + m)$, где $m$ – число подходящих слов.

__Задание 2 (1 балл).__ Допишите префиксное дерево для поиска слов по префиксу. Ваше дерево должно работать за $O(n + m)$ операции, в противном случае вы не получите баллов за это задание.

In [44]:
from typing import List

class PrefixTreeNode:
    def __init__(self):
        
        # словарь с буквами, которые могут идти после данной вершины
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False  # является ли текущая вершина концом слова

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        
        # заполняем префиксное дерево
        
        for word in vocabulary:
            self.insert_word(word)

    def insert_word(self, word: str):
        """
        Вставляет слово в префиксное дерево
        """
        node = self.root
        
        for char in word:
            if char not in node.children:
                node.children[char] = PrefixTreeNode()
                
            node = node.children[char]
            
        node.is_end_of_word = True

    def collect_words(self, node: PrefixTreeNode, prefix: str, results: List[str]):
        """
        Рекурсивно собирает все слова, начинающиеся с данного префикса
        """
        if node.is_end_of_word:
            results.append(prefix)
        for char, child_node in node.children.items():
            self.collect_words(child_node, prefix + char, results)

    def search_prefix(self, prefix: str) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """
        node = self.root
        
        # проходим по дереву по символам префикса
        for char in prefix:
            
            if char not in node.children:
                return []  # если префикс не найден
                
            node = node.children[char]
        
        # Если префикс найден, собираем все слова, начинающиеся с этого префикса
        
        results = []
        self.collect_words(node, prefix, results)
        
        return results


In [45]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его встречаемости в корпусе.

__Задание 3 (1 балл).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [46]:
class WordCompletor:
    
    def __init__(self, corpus: List[str]):
        """
        corpus: list – корпус текстов
        """
        # подсчитываем частоту каждого слова в корпусе

        self.word_frequency = Counter(list(itertools.chain.from_iterable(corpus))) 

        vocabulary = list(self.word_frequency.keys())

        self.prefix_tree = PrefixTree(vocabulary)
    

    def get_words_and_probs(self, prefix: str) -> Tuple[List[str], List[float]]:
        """
        Возвращает список слов, начинающихся на prefix, с их вероятностями
        (нормировать ничего не нужно)
        """

        words = self.prefix_tree.search_prefix(prefix)
        
        total = sum(self.word_frequency.values())
        
        probs = [self.word_frequency[word]/total if total > 0 else 0 for word in words]

        return words, probs


In [47]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему несколько следующих слов с учетом дописанного. Для этого мы воспользуемся n-граммами и будем советовать n следующих слов. Но сперва нужно получить n-граммную модель.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

Тогда, нам нужно оценить $P(w_i \mid w_{i-1}, \dots, w_{i-n})$ по частоте встречаемости n-граммы.   

__Задание 4 (1 балл).__ Напишите класс для n-граммной модели. Понятное дело, никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [48]:
class NGramLanguageModel:
    def __init__(self, corpus: List[str], n: int):

        """
        corpus: список предложений (каждое предложение представлено как строка слов)
        n: размер n-грамм
        """

        self.n = n
        self.ngram_next_counts = defaultdict(Counter)
        self.ngram_counts = Counter()
        
        # строим n-граммную модель
        # для каждого предложения (текста) из корпуса
        # генерируем всевозможные Н-граммы, добавляя их в словарь
        # и подсчитывая их частоту
        
        for sent in corpus:
            ngrams, ngrams_w_next = self.make_ngrams(sent, n)
            for ngram, ngram_next in zip(ngrams, ngrams_w_next):
                self.ngram_next_counts[ngram][ngram_next[-1]] += 1
                self.ngram_counts[ngram] += 1


    def make_ngrams(self, sentence: str, n: int):

        """
        Генерирует n-граммы из предложения.
        Возвращает список n-грамм и список предшествующих (n-1) элементов.
        """
        # для того, чтобы Н-граммы только с первым словом могли генерироваться, заполняем  n-1 позиций
        # предложения начальными токенами
        # и я добавила конечный токен на всякий случай (чтобы мы могли отслеживать, какое слово встречается в конце чаще всего)

        sentence = (n-1) * ['<PAD>'] + sentence + ['.']
        ngrams = []
        ngrams_w_next = []

        for i in range(n - 1, len(sentence)-1):
            
            preceding = sentence[i - n + 1:i+1]  # предыдущие (n-1) слова и текущее n слово (то есть n-грамма)
            word = sentence[i+1]  # следующее слово (его сохраняем, чтобы в явном виде отслеживать частотность после n-граммы)
            
            ngrams_w_next.append(tuple([*preceding, word])) # тут n-граммы со следующим словом
            ngrams.append(tuple(preceding)) # тут просто n-граммы (ключи следующих слов)
        
        return ngrams, ngrams_w_next

    def get_next_words_and_probs(self, prefix: List[str]) -> Tuple[List[str], List[float]]:
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов.
        prefix: список предыдущих (n-1) слов.
        """
        # превращаем префикс в tuple (предыдущие n-1 слов)
        if len(prefix) <= self.n-1:
            prefix_tuple = tuple(prefix) # если префикс это не n-грамма (не хватает слов), то он полностью участвует в предсказании
        else:
            prefix_tuple = tuple(prefix[-(self.n):]) # если в префиксе слов не меньше, чем n, то от него берем только последнюю n-грамму

        if prefix_tuple not in self.ngram_counts:
            return [], []  # если префикс не найден
        
        # получаем счетчики следующих слов
        next_word_counts = self.ngram_next_counts[prefix_tuple]
        total_count = self.ngram_counts[prefix_tuple]
        
        next_words = [word for word in next_word_counts.keys()]
        probs = [count/total_count for count in next_word_counts.values()]
        
        return next_words, probs

In [49]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (1 балл).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. Если вы справитесь, то сможете можете добавить опцию поддержки нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [50]:
class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """
        suggestions = []
        
        # дописывание слова
        
        string_options, string_probs = self.word_completor.get_words_and_probs(text[-1])
        

        # отсортировала по вероятностям

        string_zip = sorted(zip(string_probs, string_options), reverse = True)

        if len(string_zip) > 0:

            best_string = string_zip[0][1] # если мы вообще что-то предсказали как продолжение, берем самое вероятное слово
            
        else:
            best_string = text[-1] # если не находится продолжений для слова, просто оставляем его как было

        text[-1] = best_string # наш жадный поиск начинается с того, что мы берем наше дописанное слово как последнее для предсказания
        
        for i in range(n_words):

            
            sentence_options, sentence_probs = self.n_gram_model.get_next_words_and_probs(text)
            sentence_zip = sorted(zip(sentence_probs, sentence_options), reverse = True)

            if len(sentence_zip) > 0:
                best_sentence = ""
                best_sentence = sentence_zip[0][1] # если что-то находим - берем самое вероятное продолжение

            else:
                break
            text.append(best_sentence) # добавляем самое вероятное продолжение, чтобы от него жадным способом искать следующие

        suggestions.append(list(text[-(n_words+1):])) # из добавленных слов в текст забираем только предсказанные

        return suggestions

In [51]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [52]:
my_corpus = emails.iloc[:1000,10]

In [53]:
my_corpus

0                              [here, is, our, forecast]
1      [traveling, to, have, a, business, meeting, ta...
2                        [test, successful, way, to, go]
3      [randy, can, you, send, me, a, schedule, of, t...
4                      [let, s, shoot, for, tuesday, at]
                             ...                        
995    [jacques, still, trying, to, close, the, loop,...
996    [larrry, i, realize, you, are, disappointed, a...
997    [phillip, how, are, you, and, how, is, everyon...
998    [jacques, i, think, we, reached, an, agreement...
999    [let, s, talk, to, matt, about, the, forecast,...
Name: message_tokenized, Length: 1000, dtype: object

In [54]:
n_gram_model1 = NGramLanguageModel(corpus=my_corpus, n=2)

In [56]:
word_completor1 = WordCompletor(my_corpus)

In [57]:
text_suggestion1 = TextSuggestion(word_completor1, n_gram_model1)

In [58]:
text_suggestion1.suggest_text(['here', 'is'], n_words=3, n_texts=1)

[['is', 'a', 'spreadsheet', 'that']]

# Отчет в конце ноутбука, после второй части

## Часть 2

Настало время довести вашу систему до ума. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным, __кроме использования дополнительных данных__. Главное – вы должны обернуть вашу систему в пользовательский интерфейс с помощью [reflex](https://github.com/reflex-dev/reflex). В нем можно реализовать почти любой функционал по вашему желанию.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

При сдаче решения прикрепите весь ваш __код__, __отчет__ по второй части и __видео__ с демонстрацией работы вашей системы. Удачи!

Какие я вижу способы улучшения того, что у нас есть сейчас?

1. Наличие редких слов и н-грамм замедляет работу нашей программы, занимает дополнительное место, при этом редкие слова практически никогда не будут использоваться, так как наш выбор основан на частотности н-грам. То есть, их нужно убрать, по возможности

2. Промпт должен обрабатываться должным образом (мы не знаем, что пользователь захочет ввести в строку ввода, может, это будут цифры или знаки пунктуации)

3. Оптимальный подбор n в n-граммах (посчитать TF-IDF для всех n-грамм ?)

4. Предсказывание нескольких вариантов продолжения

5. Еще можно как-то отбирать наиболее информативные n-граммы, пытаться оставлять только те, что сильно улучшают качество и повышают вероятность комбинации слов, выделять отдельные формулы подсчета вероятности, чтобы на их основе отбирать н-граммы, а не просто жадным способом. Но это, как мне кажется, уже более трудозатратная задача, так что оставим ее релаизацию на потом

Попробуем реализовать что-то из этого

In [83]:
# в новом классе я отбрасываю все н-граммы, которые встречаются только 1 раз

class NGramLanguageModel_new:
    def __init__(self, corpus: List[str], n: int):

        """
        corpus: список предложений (каждое предложение представлено как строка слов)
        n: размер n-грамм
        """

        self.n = n
        self.ngram_next_counts = defaultdict(Counter)
        self.ngram_counts = Counter()
        
        # строим n-граммную модель
        # для каждого предложения (текста) из корпуса
        # генерируем всевозможные Н-граммы, добавляя их в словарь
        # и подсчитывая их частоту
        
        for sent in corpus:
            ngrams, ngrams_w_next = self.make_ngrams(sent, n)
            for ngram, ngram_next in zip(ngrams, ngrams_w_next):
                self.ngram_next_counts[ngram][ngram_next[-1]] += 1
                self.ngram_counts[ngram] += 1

        # убираем редкие n-граммы
        self.ngram_counts = {x: count for x, count in self.ngram_counts.items() if count > 1}


    def make_ngrams(self, sentence: str, n: int):

        """
        Генерирует n-граммы из предложения.
        Возвращает список n-грамм и список предшествующих (n-1) элементов.
        """
        # для того, чтобы Н-граммы только с первым словом могли генерироваться, заполняем  n-1 позиций
        # предложения начальными токенами
        # и я добавила конечный токен на всякий случай (чтобы мы могли отслеживать, какое слово встречается в конце чаще всего)

        sentence = (n-1) * ['<PAD>'] + sentence + ['.']
        ngrams = []
        ngrams_w_next = []

        for i in range(n - 1, len(sentence)-1):
            
            preceding = sentence[i - n + 1:i+1]  # предыдущие (n-1) слова и текущее n слово (то есть n-грамма)
            word = sentence[i+1]  # следующее слово (его сохраняем, чтобы в явном виде отслеживать частотность после n-граммы)
            
            ngrams_w_next.append(tuple([*preceding, word])) # тут n-граммы со следующим словом
            ngrams.append(tuple(preceding)) # тут просто n-граммы (ключи следующих слов)
        
        return ngrams, ngrams_w_next

    def get_next_words_and_probs(self, prefix: List[str]) -> Tuple[List[str], List[float]]:
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов.
        prefix: список предыдущих (n-1) слов.
        """
        # превращаем префикс в tuple (предыдущие n-1 слов)
        if len(prefix) <= self.n-1:
            prefix_tuple = tuple(prefix) # если префикс это не n-грамма (не хватает слов), то он полностью участвует в предсказании
        else:
            prefix_tuple = tuple(prefix[-(self.n):]) # если в префиксе слов не меньше, чем n, то от него берем только последнюю n-грамму

        if prefix_tuple not in self.ngram_counts:
            return [], []  # если префикс не найден
        
        # получаем счетчики следующих слов
        next_word_counts = self.ngram_next_counts[prefix_tuple]
        total_count = self.ngram_counts[prefix_tuple]
        
        next_words = [word for word in next_word_counts.keys()]
        probs = [count/total_count for count in next_word_counts.values()]
        
        return next_words, probs

In [77]:
import copy

# в новом классе подсказчика я реализую предсказание второго лучшего продолжения текста
# (на первой итерации выбираем второе по вероятности слово)

class TextSuggestion_new:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """
        # prompt preprocessing!
        
        text = [t.lower() for t in text]
        text = [re.sub('\d+', '', t) for t in text]
        text = [t.replace('\n', ' ') for t in text]
        text = [re.sub(r'[^a-zA-Z\s]', ' ', t) for t in text]
        
        suggestions = []
        
        # дописывание слова
        
        string_options, string_probs = self.word_completor.get_words_and_probs(text[-1])
        

        # отсортировала по вероятностям

        string_zip = sorted(zip(string_probs, string_options), reverse = True)

        if len(string_zip) > 0:

            best_string = string_zip[0][1] # если мы вообще что-то предсказали как продолжение, берем самое вероятное слово
            
        else:
            best_string = text[-1] # если не находится продолжений для слова, просто оставляем его как было

        text[-1] = best_string # наш жадный поиск начинается с того, что мы берем наше дописанное слово как последнее для предсказания
        text_2 = copy.deepcopy(text)
        for i in range(n_words):

            
            sentence_options, sentence_probs = self.n_gram_model.get_next_words_and_probs(text)
            sentence_zip = sorted(zip(sentence_probs, sentence_options), reverse = True)

            if len(sentence_zip) > 0:
                best_sentence = ""
                best_sentence = sentence_zip[0][1] # если что-то находим - берем самое вероятное продолжение

            else:
                break
            text.append(best_sentence) # добавляем самое вероятное продолжение, чтобы от него жадным способом искать следующие

        suggestions.append(list(text[-(n_words+1):])) # из добавленных слов в текст забираем только предсказанные

        # добавляем второе предсказание: на первой итерации выбираем второй наиболее вероятный вариант,
        # а на следующих - первые, все так же
        
        for i in range(n_words):

            
            sentence_options, sentence_probs = self.n_gram_model.get_next_words_and_probs(text_2)
            sentence_zip = sorted(zip(sentence_probs, sentence_options), reverse = True)

            if len(sentence_zip) > 0 and i == 0:
                best_sentence = ""
                best_sentence = sentence_zip[1][1] 
            elif len(sentence_zip) > 0 and i > 0:
                best_sentence = ""
                best_sentence = sentence_zip[0][1]
            else:
                break
            text_2.append(best_sentence) 

        suggestions.append(list(text_2[-(n_words+1):])) 

        # аналогичное можно проделать с большим количеством предложений и повторить для подсказки слов

        return suggestions

In [81]:
my_corpus = emails.iloc[:1000,10]
my_corpus
n_gram_model1 = NGramLanguageModel_new(corpus=my_corpus, n=2)
word_completor1 = WordCompletor(my_corpus)
text_suggestion1 = TextSuggestion_new(word_completor1, n_gram_model1)

In [82]:
text_suggestion1.suggest_text(['i', 'want'], n_words=3, n_texts=1)

[['want', 'to', 'be', 'a'], ['want', 'your', 'print', 'out']]

Полный код проекта, через который я запускала приложение из видео:

In [ ]:
import reflex as rx

import pandas as pd
from typing import Union, List, Tuple
import re
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
import itertools

emails = pd.read_csv('emails.csv')
my_corpus = emails.iloc[:50,:]

import sys
sys.path.append('/Users/alina/NGramModel/')
from rxconfig import config

class PrefixTreeNode:

    def __init__(self):

        self.children = {}
        self.is_end_of_word = False 

class PrefixTree:
    def __init__(self, vocabulary):

        self.root = PrefixTreeNode()
        for word in vocabulary:
            self._insert(word)

    def _insert(self, word: str):

        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = PrefixTreeNode()
            node = node.children[char]
        node.is_end_of_word = True

    def _collect_words(self, node, prefix, results):

        if node.is_end_of_word:
            results.append(prefix)
        for char, child_node in node.children.items():
            self._collect_words(child_node, prefix + char, results)

    def search_prefix(self, prefix):

        node = self.root

        for char in prefix:
            if char not in node.children:
                return [] 
            node = node.children[char]
        
   
        results = []
        self._collect_words(node, prefix, results)
        return results

class WordCompletor:
    def __init__(self, corpus):

        self.word_frequency = Counter(list(itertools.chain.from_iterable(corpus))) 

        vocabulary = list(self.word_frequency.keys())
    
        self.prefix_tree = PrefixTree(vocabulary)

    def get_words_and_probs(self, prefix: str):


        words = self.prefix_tree.search_prefix(prefix)
        
        total = sum(self.word_frequency.values())
        
        probs = [self.word_frequency[word]/total if total > 0 else 0 for word in words]

        return words, probs
    

class NGramLanguageModel:
    def __init__(self, corpus, n):

        self.n = n
        self.ngram_next_counts = defaultdict(Counter)
        self.ngram_counts = Counter()
        
        for sent in corpus:
            ngrams, ngrams_w_next = self.make_ngrams(sent, n)
            for ngram, ngram_next in zip(ngrams, ngrams_w_next):
                self.ngram_next_counts[ngram][ngram_next[-1]] += 1
                self.ngram_counts[ngram] += 1


    def make_ngrams(self, sentence, n):

        sentence = (n-1) * ['<PAD>'] + sentence + ['.']
        ngrams = []
        ngrams_w_next = []

        for i in range(n - 1, len(sentence)-1):
            preceding = sentence[i - n + 1:i+1] 
            word = sentence[i+1]
            ngrams_w_next.append(tuple([*preceding, word]))
            ngrams.append(tuple(preceding))
        
        return ngrams, ngrams_w_next
    
    def get_next_words_and_probs(self, prefix):

        if len(prefix) <= self.n-1:
            prefix_tuple = tuple(prefix)
        else:
            prefix_tuple = tuple(prefix[-(self.n):])

        if prefix_tuple not in self.ngram_counts:
            return [], []
        
        next_word_counts = self.ngram_next_counts[prefix_tuple]
        total_count = self.ngram_counts[prefix_tuple]
        
        next_words = [word for word in next_word_counts.keys()]
        probs = [count/total_count for count in next_word_counts.values()]
        
        return next_words, probs

class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text, n_words=3, n_texts=1):

        suggestions = []

    
        string_options, string_probs = self.word_completor.get_words_and_probs(text[-1])


        string_zip = sorted(zip(string_probs, string_options), reverse = True)
        
        if len(string_zip) > 0:

            best_string = string_zip[0][1]
        else:
            best_string = text[-1]

        text[-1] = best_string

        for i in range(n_words):
            
            sentence_options, sentence_probs = self.n_gram_model.get_next_words_and_probs(text)
            sentence_zip = sorted(zip(sentence_probs, sentence_options), reverse = True)

            if len(sentence_zip) > 0:
                best_sentence = ''
                best_sentence = sentence_zip[0][1]

            else:
                break
            text.append(best_sentence)

        suggestions.append(list(text[-(n_words+1):]))

        return suggestions


my_corpus['message_wo_metadata'] = my_corpus['message'].apply(lambda x: x.split('\n\n', 1)[1])



def split_forward(msg):
    
    if 'Subject:' in msg:
        return msg.rsplit('Subject:', 1)[-1].split('\n\n', 1)[-1]
        
    else:
        return msg
       
my_corpus['message_wo_forward'] = my_corpus['message_wo_metadata'].apply(lambda x: split_forward(x))
my_corpus['message_wo_meetings'] = my_corpus['message_wo_forward'].apply(lambda x: x.split('----------------------', 1)[0])
my_corpus['message_wo_subscription_metadata'] = my_corpus['message_wo_meetings'].apply(lambda x: split_forward(x))


my_corpus['message_wo_emails'] = my_corpus['message_wo_subscription_metadata'].apply(lambda x: re.sub(r'\S*@\S*\s?', '', x))
my_corpus['message_wo_urls'] = my_corpus['message_wo_emails'].apply(lambda x: re.sub(r'http\S+', '', x))
my_corpus['message_wo_files'] = my_corpus['message_wo_urls'].apply(lambda x: re.sub(r'-\s\S*.\S*', '', x))

def preprocess(x):

    x_lowercase = x.lower()
    x_no_digits = re.sub('\d+', '', x_lowercase)
    x_no_nextstring = x_no_digits.replace('\n', ' ')
    x_no_punctuation = re.sub(r'[^a-zA-Z\s]', ' ', x_no_nextstring)
    x_final = re.sub(' +', ' ', x_no_punctuation)

    return x_final

my_corpus['message_preprocessed'] = my_corpus['message_wo_files'].apply(lambda x: preprocess(x))
my_corpus['message_tokenized'] = my_corpus['message_preprocessed'].apply(lambda x: x.split())

word_completor = WordCompletor(my_corpus.iloc[:,10])
n_gram_model = NGramLanguageModel(corpus=my_corpus.iloc[:,10], n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)


class State(rx.State):

    prompt = ""
    suggested_text = ""
    suggested_text_2 = ""
    processing = False
    complete = False

    def get_suggestion(self):
        if self.prompt == "":
            return rx.window_alert("Please, enter the prompt!")
        self.processing, self.complete = True, False
        yield
        response = text_suggestion.suggest_text(list(self.prompt.split()), n_words=3, n_texts=1)
        self.suggested_text = ' '.join(response[0])
        self.processing, self.complete = False, True


def index():
    return rx.center(
        rx.vstack(
            rx.heading("Text suggestion online!", size = "8",),
            rx.heading(
            "Made by Grushina Daria, HSE-NES Joint Programme",
            size="5",),
            rx.input(
                placeholder="Enter a prompt... ",
                on_blur=State.set_prompt,
                width="25em",
                border_color="#1c2024",
            ),
            rx.button(
                "suggest continuation", 
                on_click=State.get_suggestion,
                width="25em",
                loading=State.processing, 
                background_color="#1c2024"
            ),
            rx.cond(
                State.complete,
                rx.text(State.suggested_text, text_align="center", font_weight="bold", color="black")
            ),
            align="center",
        ),
        width="100%",
        height="100vh",
        background="linear-gradient(to right, #a8c0ff, #3f2b96)"
    )

app = rx.App()
app.add_page(index, title="Text suggestion online!")


### Отчет

Итак, мы сделали полноценную пользовательскую систему продолжения текста!

Ее основа базируется на N-граммной модели: мы подсчитываем все N-граммы в нашем корпусе текстов, отмечая, как часто после каждой из них встречаются те или иные слова (для скорости пользуемся префиксным деревом). Далее, мы считаем вероятности (деля количество раз, когда N-грамма продолжилась определенным словом, на все количество вхождений данной N-граммы), сортируем наши продолжения N-грамм по ним, и жадным способом на каждой итерации (при предсказании каждого последующего слова) ищем самое вероятное продолжение. Продолжения слов мы дописываем, ища самое частотное слово с таким же началом в словаре.

В качестве корпуса слов мы используем датасет электронных писем, состоящий из 500 000 сообщений (предварительно предобработанный, очищенный от метаданных, цифр, пунктуации, иных спец. символов).

После мы оборачиваем нашу систему в пользовательский интерфейс при помощи фреймворка reflex: у нас получился сайт, где пользователь может ввести в строку последовательность слов и получить нажатием кнопки наиболее вероятное продолжение.

По инструкции reflex, я отредактировала дефолтный State класс, который согласно функционалу нашего приложения подает на выход самый вероятный ответ подсказчика текста. Со стороны фронта через функцию index создала привлекательный дизайн, приятный глазу и понятный пользователю. 

В качестве улучшения базового варианта системы мной были предложены и реализованы: фильтрация редких н-грамм (отбрасывание тех, что встречаются меньше 2 раз), подсказка более одного варианта продолжения текста посредством выбора второго лучшего варианта на первой итерации предсказания, подходящая предобработка промпта (предобработка, аналогичная предобработке входных данных).

С какими проблемами я столкнулась и какие перспективы их решения существуют?

1. На локальном сервере в режиме реального времени невозможно обрабатывать большие количества данных. Это сильно снижает предсказательные способности нашей системы, потому что сервер просто падает при попытке предобработать входные данные. Я тестировала систему на маленьком кусочке датасета, чтобы она была работоспособна и относительно быстро отвечала. Это вопрос ограничения производительности оборудования

2. Конечно, жадный метод выбора продолжений не слишком оптимален. Хотелось бы заменить его максимизацией вероятности/качества/отбором наиболее информативных n-грамм в будущем (entropy-based)
   
   
4. Так же я не смогла реализовать вывод произвольного количества предсказаний во фронт. Конечно, это сложно и с точки зрения технической (самой поддержки фронта, лично для меня было подвигом выводить даже одно предсказание, не то что несколько :( ), и с точки зрения написания самого алгоритма. Следствие из жадного метода - я делала это итеративными предсказаниями вручную, а было бы здорово, если бы наша система умела в зависимости от того, сколько связанных со словом/предложением н-грамм есть, выдавать разное количество предсказаний (по трешхолду вероятности или по количеству вхождений n-граммы и ее продолжения), или даже в режиме реального времени мочь их изменять (количество и само содержание). Но я на данном этапе даже не представляю, как можно было бы это реализовать, поэтому считаю, что это тема для отдельного маленького проекта

5. Конечно, в контексте продолжения слова, нашей голубой мечтой было бы, чтобы мы как-то реагировали даже на те слова, которые в нашем корпусе слов отсутствуют, потому что сейчас наша система вообще такие случаи не умеет обрабатывать и просто выдает пустой вывод. Это можно сделать либо расширением данных (хотя это вопроса полностью не решит, так как есть новый появляющийся сленг, и данные не будут успевать обновляться, есть опечатки), либо предсказанием n-грамм внутри слова (уже внутри слова смотреть какие сочетания букв самые популярные и пытаться достроить слово), либо fuzzy string matching - по какой-нибудь метрике (cosine, jaccard) искать на какое слово больше всего похоже введенное, мне кажется последнее было бы самым оптимальным.

6. Оптимизация гиперпараметров. Я не подбирала n и количество возвращаемых слов, но было бы здорово измерить и понять, какие дают лучшее качество (во втором случае мы бы измеряли когда добавление новых слов начинает ухудшать качество и тогда бы переставали их добавлять, либо просто ориентировались бы на трешхолд). Но здесь, конечно, отдельной задачей стоит сам выбор функционала качества, так что задача не так проста, как кажется

7. Другие аспекты оптимизации: более широкий функционал приложения, учет каких-то краевых случаев (а что, если у нас вообще нет продолжений, а что, если нет второго/третьего продолжения, а что, если пользователь введет слово на русском или что-то другое нетипичное и так далее), использование большего количества данных
